<a href="https://colab.research.google.com/github/RGologorsky/7tCVO/blob/main/Copy_of_DI_Podcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
url0 = "https://divineinterventionpodcasts.com/"
urls = [url0]

In [ ]:
url1 = "https://divineinterventionpodcasts.com/category/shelf-exams/neurology-shelf/" # page 1, 2, 3
url2 = "https://divineinterventionpodcasts.com/category/shelf-exams/medicine-shelf/" # page 1-8
url3 = "https://divineinterventionpodcasts.com/category/shelf-exams/pediatrics-shelf/" # page 1-8
url4 = "https://divineinterventionpodcasts.com/category/shelf-exams/obstetrics-and-gynecology-shelf/" # page 1-8
url5 = "https://divineinterventionpodcasts.com/category/shelf-exams/surgery-shelf/" # page 1-8
url6 = "https://divineinterventionpodcasts.com/category/shelf-exams/psychiatry-shelf/"

urls = [url1, url2, url3, url4, url5, url6]

In [ ]:
import os
import urllib.request
import requests
from bs4 import BeautifulSoup
import unicodedata
from pathlib import Path
from pprint import pprint
import re

In [ ]:
! pip install mutagen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00


In [ ]:
from mutagen.mp3 import MP3


In [ ]:
def get_titles_inner(page, url):
  page_url = f"{url}/page/{page}"
  html_content = requests.get(page_url).text
  soup = BeautifulSoup(html_content, "lxml")

  mydivs = soup.find_all("header", {"class": "entry-header"})
  #titles = [div.h1.a.get_text(strip=True).strip() for div in mydivs]
  #mp3s   = [div.findNext("div", {"class": "entry-content"}).audio.source['src'] for div in mydivs]
  #return titles, mp3s
  titles = []
  mp3s   = []
  for div in mydivs:
    title = div.h1.a.get_text(strip=True)
    content = div.findNext("div", {"class": "entry-content"})

    try:
      audio_src = content.audio.source['src']
    except:
      hrefs = [a["href"] for a in content.findAll('a')]
      audio_src = hrefs

    #print(audio_src)

    titles.append(title)
    mp3s.append(audio_src)

  return titles, mp3s

def get_titles(url):
  all_titles = []
  all_mp3s   = []
  page = 1
  while True:
    new_titles, new_mp3s = get_titles_inner(page, url)
    if len(new_titles) > 0:
      page += 1
      all_titles += new_titles
      all_mp3s   += new_mp3s
    else:
      #print("Page ", page)
      break
  return all_titles, all_mp3s

In [ ]:
all_titles = []
all_mp3s   = []

for url in urls:
  new_titles, new_mp3s = get_titles(url)
  all_titles += new_titles
  all_mp3s   += new_mp3s

In [ ]:
d  = dict(zip(all_titles, all_mp3s))
d2 = {mp3:title for mp3, title in zip(all_mp3s, all_titles) if isinstance(mp3, str)}

In [ ]:
len(all_mp3s)

477

In [ ]:
for mp3 in all_mp3s:
  if isinstance(mp3, list):
    print(mp3)

[]
['https://divineinterventionpodcasts.files.wordpress.com/2018/09/divine-intervention-episode-51-comprehensive-step-1-biochemistry-review-session-2.pdf', 'https://www.youtube.com/watch?v=JZa7Q8Wy1KI']
['https://divineinterventionpodcasts.files.wordpress.com/2018/09/divine-intervention-episode-50-comprehensive-step-1-biochemistry-review-session-1.pdf', 'https://www.youtube.com/watch?v=NiVrHaM3S8U&t=5099s']
['https://divineinterventionpodcasts.files.wordpress.com/2018/05/medicine-keywords-c.pdf', 'https://divineinterventionpodcasts.files.wordpress.com/2018/05/divine-intervention-episode-32-nbme-medicine-shelf-review-session-4.mov']
['https://divineinterventionpodcasts.files.wordpress.com/2018/05/medicine-keywords-b.pdf', 'https://www.dropbox.com/s/ek5mjqdu56xgqy8/Divine%20Intervention%20Episode%2031-NBME%20Medicine%20Shelf%20Review%20Session%203.mov?dl=0']
['https://divineinterventionpodcasts.files.wordpress.com/2018/05/nbme-medicine-shelf-review-session-2.pdf', 'https://divineinterven

In [ ]:
for mp3 in all_mp3s:
  if isinstance(mp3, str):
    name = Path(mp3).name
    if not name.startswith("episode"):
      print(name)
      print(d2[mp3])
      print("*"*50)

announcement-500mcq-qbank-june.mp3?_=1
500 MCQ NBME Style Review for Step 2/3 (June 2023)
**************************************************
announcement-april-classes.mp3?_=3
April/Early May Classes (Step 1, Step 2/3, Test Taking Strategies, Ethics/Social Sciences, and Biostatistics)
**************************************************
ep-383-clutch-smoking.mp3?_=4
Divine Intervention Episode 383 – The Clutch Smoking Podcast (for Step 1-3)
**************************************************
monday-motivation-dip-2022.mp3?_=4
Motivation Monday – How To Use The Divine Intervention Podcasts in 2022 (Shelf Exams, Step 2CK/3)
**************************************************
dsd-review-podcast.mp3?_=1
Divine Intervention Episode 357 – The Clutch Disorders of Sexual Differentiation Review (for Step 1-3)
**************************************************
psych-pharm-review.mp3?_=9
Divine Intervention Episode 349 – The Clutch Psych Pharm Review (for the psych shelf, Step 2CK, and Step 3).
*****

In [ ]:
def notkeep(s,t): 
  name = Path(s).name
  return isinstance(name, str) and (name.startswith("announcement") or t.startswith("Announcement"))
notkeeplist = [f"{title}: {mp3}" for mp3, title in d2.items() if isinstance(mp3, str) and notkeep(mp3, title)]
print(len(notkeeplist), *notkeeplist, sep="\n")

d2_subset = {mp3:title for mp3,title in d2.items() if isinstance(mp3, str) and not notkeep(mp3, title)}
print(len(d2_subset))

16
500 MCQ NBME Style Review for Step 2/3 (June 2023): https://divineinterventionpodcasts.files.wordpress.com/2023/03/announcement-500mcq-qbank-june.mp3?_=1
April/Early May Classes (Step 1, Step 2/3, Test Taking Strategies, Ethics/Social Sciences, and Biostatistics): https://divineinterventionpodcasts.files.wordpress.com/2023/03/announcement-april-classes.mp3?_=3
Announcement: November Step 2CK/3 (and COMLEX) Review Course  + NBME Test Taking Strategies Course: https://divineinterventionpodcasts.files.wordpress.com/2021/10/announcement-november-.mp3?_=4
Announcement: October NBME Test Taking Course and Step 2CK/3/COMLEX Review Course: https://divineinterventionpodcasts.files.wordpress.com/2021/09/announcement_-october-courses.mp3?_=2
Announcement: September USMLE Step 2CK/3 Course + NBME Test Taking Strategies Course (9/6-9/9): https://divineinterventionpodcasts.files.wordpress.com/2021/08/announcement-september-course-.mp3?_=6
Announcement Podcast: Upcoming NBME Test Taking Strategies

In [ ]:
title = "Divine Intervention Episode 225 – USMLE Step 2CK Rapid Review Series 35."
re.search("Episode (\d+)", title).group(1)

'225'

In [ ]:
def convert_name(mp3, title):
  name = str(Path(mp3).name)
      
  if name.startswith("psych-pharm-review"):
    return "349-clutch-psych-pharm-review"
  if name.startswith("dsd-review-podcast.mp3"):
    return "357-clutch-disorders-sexual-diff-review.mp3"
  if name.startswith("episode-88.mp3"):
    return "88-review3-meningitis.mp3"
  if name.startswith("divine-intervention-episode"):
    return name[len("divine-intervention-episode-"):]
  if name.startswith("Episode-"):
    return name[len("Episode-"):]
  if name.startswith("episode-"):
    return name[len("episode-"):]
  if name.startswith("ep-"):
    return name[len("ep-"):]

  if name.startswith("rr-"):
    n = re.search("Episode (\d+)", title).group(1)
    return f"{n}-{name}"
    
  print("Weird: ", name)
  return name

In [ ]:
new_names = [convert_name(mp3, title) for mp3, title in d2_subset.items()]

Weird:  monday-motivation-dip-2022.mp3?_=4
Weird:  motivation-monday-anxiety.mp3?_=10
Weird:  motivation-monday-personal-statements.mp3?_=3
Weird:  motivation-monday-reviewing-uworld-and-nbmes.mp3?_=5
Weird:  motivation-monday-4-week-dedicated.mp3?_=9


In [ ]:
d3 = {convert_name(mp3, title):title for mp3, title in d2_subset.items()}

# d  = dict(zip(all_titles, all_mp3s))
# d2 = {mp3:title for mp3, title in zip(all_mp3s, all_titles) if isinstance(mp3, str)}

Weird:  monday-motivation-dip-2022.mp3?_=4
Weird:  motivation-monday-anxiety.mp3?_=10
Weird:  motivation-monday-personal-statements.mp3?_=3
Weird:  motivation-monday-reviewing-uworld-and-nbmes.mp3?_=5
Weird:  motivation-monday-4-week-dedicated.mp3?_=9


In [ ]:
for nn in new_names:
  print(nn)
  print(d3[nn])
  print(d[d3[nn]])
  print("*"*50)

454-rr-94.mp3?_=2
Divine Intervention Episode 454: USMLE Step 2/3 Rapid Review Series 94
https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-454-rr-94.mp3?_=2
**************************************************
453-psychiatry-timelines.mp3?_=4
Divine Intervention Episode 453: A Quick Review of HY Psychiatry Timelines
https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-453-psychiatry-timelines.mp3?_=4
**************************************************
452-mh-and-nms.mp3?_=5
Divine Intervention Episode 452: NMS and Malignant Hyperthermia (vs Serotonin Syndrome)
https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-452-mh-and-nms.mp3?_=5
**************************************************
451-serotonin-syndrome-.mp3?_=6
Divine Intervention Episode 451: Serotonin Syndrome (with integrations)
https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-451-serotonin-syndrome-.mp3?_=6
*********************************************

In [ ]:
re.search("\d+", "447_-divine-2023.mp3?_=10").group()

'447'

In [ ]:
# add 0
def add0(name):
  if name.startswith("motivation-monday"):
    return name
    
  n = int(re.search("\d+", name).group(0)) #int(name[:name.index("-")])
  if n < 10:
    return "00" + name
  if n < 100:
    return "0" + name
  return name

new_names2 = [add0(name) for name in new_names]

In [ ]:
sorted(new_names2)

['001.mp3?_=6',
 '002-viral-cases-a.mp3?_=5',
 '003-viral-cases-b.mp3?_=4',
 '004-hematology-a.mp3?_=3',
 '005-cardio-a-physiology.mp3?_=2',
 '006-20-in-about-20-pharmacology-cases.mp3?_=1',
 '007-viral-cases-c-and-fungal-cases.mp3?_=10',
 '008-comprehensive-heme-pharmacology.mp3?_=9',
 '009-studying-for-step-2ck.mp3?_=8',
 '010-comprehensive-gi-pharmacology.mp3?_=7',
 '011-glycogen-and-lysosomal-storage-diseases.mp3?_=6',
 '012-bacterial-cases-a-2.mp3?_=5',
 '013-metabolism-a.mp3?_=4',
 '014-comprehensive-immuno-pharmacology-and-the-monoclonal-alphabet.mp3?_=3',
 '015-metabolism-b.mp3?_=2',
 '016-bacterial-cases-b.mp3?_=1',
 '017-diseases-of-the-pediatric-population-part-1.mp3?_=6',
 '018-autonomic-pharmacology-part-1.mp3?_=5',
 '019-neuro-shelf-review-part-1.mp3?_=4',
 '020-autonomic-pharmacology-part-2.mp3?_=3',
 '025-autonomic-pharmacology-part-3.mp3?_=2',
 '026-usmle-radiology.mp3?_=1',
 '027-antibiotics-part-1.mp3?_=6',
 '028-antibiotics-part-2.mp3?_=5',
 '033-antibiotics-part-3-

In [ ]:
print(len(new_names2))

449


In [ ]:
len(d2_subset)

449

In [ ]:
#d2_subset.items() mp3:title
#d3 = {convert_name(mp3, title):title for mp3, title in d2_subset.items()}
#d4 = {add0(convert_name(mp3, title)):title for name, title in d2_subset.items()}

#d5 = {mp3:}

In [ ]:
d4 = {mp3:add0(convert_name(mp3,title)) for mp3, title in d2_subset.items()}
print("len", len(d4))
print(*sorted(list(d4.values())), sep="\n")

Weird:  monday-motivation-dip-2022.mp3?_=4
Weird:  motivation-monday-anxiety.mp3?_=10
Weird:  motivation-monday-personal-statements.mp3?_=3
Weird:  motivation-monday-reviewing-uworld-and-nbmes.mp3?_=5
Weird:  motivation-monday-4-week-dedicated.mp3?_=9
len 449
001.mp3?_=6
002-viral-cases-a.mp3?_=5
003-viral-cases-b.mp3?_=4
004-hematology-a.mp3?_=3
005-cardio-a-physiology.mp3?_=2
006-20-in-about-20-pharmacology-cases.mp3?_=1
007-viral-cases-c-and-fungal-cases.mp3?_=10
008-comprehensive-heme-pharmacology.mp3?_=9
009-studying-for-step-2ck.mp3?_=8
010-comprehensive-gi-pharmacology.mp3?_=7
011-glycogen-and-lysosomal-storage-diseases.mp3?_=6
012-bacterial-cases-a-2.mp3?_=5
013-metabolism-a.mp3?_=4
014-comprehensive-immuno-pharmacology-and-the-monoclonal-alphabet.mp3?_=3
015-metabolism-b.mp3?_=2
016-bacterial-cases-b.mp3?_=1
017-diseases-of-the-pediatric-population-part-1.mp3?_=6
018-autonomic-pharmacology-part-1.mp3?_=5
019-neuro-shelf-review-part-1.mp3?_=4
020-autonomic-pharmacology-part-2.m

In [ ]:
d2_subset.items()

dict_items([('https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-454-rr-94.mp3?_=2', 'Divine Intervention Episode 454: USMLE Step 2/3 Rapid Review Series\xa094'), ('https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-453-psychiatry-timelines.mp3?_=4', 'Divine Intervention Episode 453: A Quick Review of HY Psychiatry\xa0Timelines'), ('https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-452-mh-and-nms.mp3?_=5', 'Divine Intervention Episode 452: NMS and Malignant Hyperthermia (vs Serotonin\xa0Syndrome)'), ('https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-451-serotonin-syndrome-.mp3?_=6', 'Divine Intervention Episode 451: Serotonin Syndrome (with\xa0integrations)'), ('https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-450-clutch-hypernatremia.mp3?_=7', 'Divine Intervention Episode 450: The Clutch Hypernatremia Podcast (for Step\xa01-3)'), ('https://divineinterventionpodcasts.files.wordpress

In [ ]:
download_loc = "gdrive/MyDrive/Divine/All" #/content/drive/MyDrive/Divine/All
os.listdir(download_loc)

['399-the-3-confusing-poisonings.mp3',
 '365-eye-2.mp3',
 '361-the-eye-1.mp3',
 '336-rr-64.mp3',
 '315-cow-podcast.mp3',
 '311-csf-brain-imaging-.mp3',
 '211-rr-32.mp3',
 '058-neurology-clerkship-shelf-review-part-7.mp3',
 '049-neurology-shelf-review-part-6.mp3',
 '048-neurology-shelf-review-part-5-1.mp3',
 '047-neurology-clerkship-shelf-review-part-4.mp3',
 '046-neurology-clerkship-shelf-review-part-3.mp3',
 '045-neurology-clerkship-review-part-2.mp3',
 '019-neuro-shelf-review-part-1.mp3',
 '440-hd-dexamethasone-.mp3',
 '430-clutch-raas-podcast.mp3',
 '429-icu-crystalloid-.mp3',
 '417-pulmonary-pathophysiology-series-8.mp3',
 '435-nmj-diseases.mp3',
 '397-pulmonary-pathophysiology-series-3.mp3',
 '392-pulmonary-pathophysiology.mp3',
 '384-ibd-.mp3',
 '367-clutch-amyloid.mp3',
 '348-the-clutch-hypercalcemia-podcast.mp3',
 '374-sc-lesions.mp3',
 '371-the-spinal-cord.mp3',
 '343-paraneoplastics.mp3',
 '339-pe.mp3',
 '333-pressors-and-inotropes.mp3',
 '298-gi-4.mp3',
 '253-starling-.mp3',

In [ ]:
download_loc = "/content/drive/MyDrive/Divine/All" #/content/drive/MyDrive/Divine/All

def download_mp3(mp3, name):
  mp3  = mp3[0:mp3.index(".mp3")] + ".mp3"  
  name = name[0:name.index(".mp3")] + ".mp3"

  fp = f"{download_loc}/{name}"
  if Path(fp).is_file():
    print("Is file")

  else:
    r = requests.get(mp3, stream = True) 
    with open(fp, "wb") as file: 
        for block in r.iter_content(chunk_size = 1024):
            if block: 
                file.write(block) 

In [ ]:
# audio = MP3(f"{download_loc}/episode-436-eye-findings-and-the-usmles.mp3")
# print(audio.info.length)
# print(audio.info.bitrate)

In [ ]:
for mp3,name in d4.items():
  #name = name[0:name.index(".mp3")] 
  print(mp3, name)
  download_mp3(mp3,name)

https://divineinterventionpodcasts.files.wordpress.com/2023/04/episode-454-rr-94.mp3?_=2 454-rr-94.mp3?_=2


FileNotFoundError: ignored

In [ ]:
1746.6320625/60

29.110534375

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
#drive.mount('/content/drive', force_remount=True)

In [ ]:
print(len(all_titles), len(all_mp3s))

27 27


In [ ]:
all_titles

['D',
 'i',
 'v',
 'i',
 'n',
 'e',
 ' ',
 'I',
 'n',
 't',
 'e',
 'r',
 'v',
 'e',
 'n',
 't',
 'i',
 'o',
 'n',
 ' ',
 'E',
 'p',
 'i',
 's',
 'o',
 'd',
 'e',
 ' ',
 '4',
 '3',
 '6',
 ':',
 ' ',
 'E',
 'y',
 'e',
 ' ',
 'F',
 'i',
 'n',
 'd',
 'i',
 'n',
 'g',
 's',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 'T',
 'h',
 'e',
 ' ',
 'U',
 'S',
 'M',
 'L',
 'E',
 's',
 ' ',
 '(',
 'f',
 'l',
 'o',
 'r',
 'i',
 'd',
 'l',
 'y',
 ' ',
 'H',
 'Y',
 ' ',
 'f',
 'o',
 'r',
 ' ',
 'S',
 't',
 'e',
 'p',
 '\xa0',
 '1',
 '-',
 '3',
 ')',
 'D',
 'i',
 'v',
 'i',
 'n',
 'e',
 ' ',
 'I',
 'n',
 't',
 'e',
 'r',
 'v',
 'e',
 'n',
 't',
 'i',
 'o',
 'n',
 ' ',
 'E',
 'p',
 'i',
 's',
 'o',
 'd',
 'e',
 ' ',
 '4',
 '3',
 '5',
 ':',
 ' ',
 'D',
 'i',
 's',
 'e',
 'a',
 's',
 'e',
 's',
 ' ',
 'o',
 'f',
 ' ',
 'T',
 'h',
 'e',
 ' ',
 'N',
 'e',
 'u',
 'r',
 'o',
 'm',
 'u',
 's',
 'c',
 'u',
 'l',
 'a',
 'r',
 ' ',
 'J',
 'u',
 'n',
 'c',
 't',
 'i',
 'o',
 'n',
 ' ',
 '(',
 'f',
 'o',
 'r',
 ' ',
 'S',
 't',
 

In [ ]:
all_titles = []
page = 1
while True:
  new_titles = get_titles(page)
  if len(new_titles) > 0:
    page += 1
    all_titles += new_titles
  else:
    print("Page ", page)
    break


Page  2


In [ ]:
pprint(all_titles)

['Divine Intervention Episode 436: Eye Findings and The USMLEs (floridly HY '
 'for Step\xa01-3)',
 'Divine Intervention Episode 349 – The Clutch Psych Pharm Review (for the '
 'psych shelf, Step 2CK, and Step\xa03).',
 'Divine Intervention Episode 166 – USMLE Step 2CK Rapid Review Series 20\xa0'
 '(Psych).',
 'Divine Intervention Episode 164 – The Clutch Toxicology Podcast (Drugs of '
 'abuse, overdose, withdrawal,\xa0toxidromes).',
 'Divine Intervention Episode 131 – USMLE Step 2CK Rapid Review Series 11 (OB, '
 'Psych, IM,\xa0Neuro)',
 'Divine Intervention Episode 68 – Psych Pharm Part 2 (Final\xa0Part)',
 'Divine Intervention Episode 67 – Psych Pharmacology (Part 1 of\xa02)',
 'Divine Intervention Episode 23 – Comprehensive 3rd Year Psychiatry Shelf\xa0'
 'Review.']


In [ ]:
def normalstring(s): return s.replace("\xa0", " ").replace("Divine Intervention ", "")
pretty_all_titles = [normalstring(s) for s in all_titles]

def format(lst):
  formatted_lst = [f"{i}. {title}" for i, title in enumerate(lst)]
  
  # Write lines to file
  name = str(Path(url).name)
  with open(f'di_titles_{name}.txt', 'w') as writefile:
      writefile.write(f"{name}\n\n")
      for line in formatted_lst:
        writefile.write(f"{line}\n")

format(pretty_all_titles)